# MySQL Pivot: rotating rows to columns
https://linuxhint.com/mysql_pivot/

In [2]:
import mysql.connector as sql
import pandas as pd

mydb = sql.connect (
    host = "localhost",
    user = "root",
    password = "ZAQs8271911c$",
    auth_plugin='mysql_native_password',
    buffered = True,          # using this for reading table, panda dont need it
    #multi=True,                 # if wanna use mulitple query
    database = "sql_fr_ai"
)
# #! create table
# mycoursor = mydb.cursor()
# sql_query = """

# CREATE TABLE if not exists column_to_row_practice
# 	(`id` int, 
#     `month` varchar(3), 
#     `col1` varchar(1), 
#     `col2` varchar(1), 
#     `col3` varchar(4), 
#     `col4` varchar(1))

#  """
# mycoursor.execute(sql_query)

In [3]:

#! check table and values

sql_query = """
select 
            *
from 
            result
"""

df = pd.read_sql_query(
    sql_query,
    mydb)


df

,std_id,course_id,mark_type,marks
0,1937463,CSE-401,Internal Exam,15
1,1937463,CSE-401,Mid Term Exam,20
2,1937463,CSE-401,Final Exam,35
3,1937464,CSE-403,Internal Exam,17
4,1937464,CSE-403,Mid Term Exam,15
5,1937464,CSE-403,Final Exam,30
6,1937465,CSE-401,Internal Exam,18
7,1937465,CSE-401,Mid Term Exam,23
8,1937465,CSE-401,Final Exam,38
9,1937466,CSE-407,Internal Exam,20


#! Rotate rows to columns using CASE statement:

In [11]:



sql_query = """
SELECT
            std_id,
            course_id,
            MAX(case when mark_type = "Internal Exam"
                                        THEN marks
                                        END ) "Internal Exam",
            MAX(case when mark_type = "Mid Term Exam"
                                        THEN marks
                                        END ) "Mid Term Exam",
            MAX(case when mark_type = "Final Exam"
                                        THEN marks
                                        END ) "Final Exam",
            SUM(marks) AS total
FROM 
            result
GROUP BY
            std_id,
            course_id
"""

df = pd.read_sql_query(
    sql_query,
    mydb)


df

,std_id,course_id,Internal Exam,Mid Term Exam,Final Exam,total
0,1937463,CSE-401,15,20,35,70.0
1,1937464,CSE-403,17,15,30,62.0
2,1937465,CSE-401,18,23,38,79.0
3,1937466,CSE-407,20,22,40,82.0


#! Rotate rows to columns using CASE statement:
#### get name of course_id from table course
#### get name of std_id from table student

In [14]:

#! check table and values

sql_query = """
select 
            *
from 
            result
"""

df = pd.read_sql_query(
    sql_query,
    mydb)


df

,std_id,course_id,mark_type,marks
0,1937463,CSE-401,Internal Exam,15
1,1937463,CSE-401,Mid Term Exam,20
2,1937463,CSE-401,Final Exam,35
3,1937464,CSE-403,Internal Exam,17
4,1937464,CSE-403,Mid Term Exam,15
5,1937464,CSE-403,Final Exam,30
6,1937465,CSE-401,Internal Exam,18
7,1937465,CSE-401,Mid Term Exam,23
8,1937465,CSE-401,Final Exam,38
9,1937466,CSE-407,Internal Exam,20


In [12]:
sql_query = """

SELECT *
FROM students
"""

df = pd.read_sql_query(
    sql_query,
    mydb)


df

,id,name,department
0,1937463,Harper Lee,CSE
1,1937464,Garcia Marquez,CSE
2,1937465,"Forster, E.M.",CSE
3,1937466,Ralph Ellison,CSE


In [13]:
sql_query = """

SELECT *
FROM courses
"""

df = pd.read_sql_query(
    sql_query,
    mydb)


df

,course_id,name,credit
0,CSE-401,Object Oriented programming,3
1,CSE-403,Data Structure,2
2,CSE-407,Unix programming,2


In [24]:
sql_query = """

SELECT
            students.name AS "name",
            courses.name AS "course",
            MAX( CASE WHEN result.mark_type = "Internal Exam"
                                        THEN result.marks
                        END) "internal mark",
            MAX( CASE WHEN result.mark_type = "Mid Term Exam"
                                        THEN result.marks
                        END) "mid term mark",
            MAX( CASE WHEN result.mark_type = "Final Exam"
                                        THEN result.marks
                        END) "final mark",
            SUM(result.marks) AS total
FROM 
            students,
            courses,
            result
WHERE
            result.std_id = students.id
            AND
            result.course_id = courses.course_id
GROUP BY
            result.std_id,
            result.course_id


        
"""

df = pd.read_sql_query(
    sql_query,
    mydb)


df

,name,course,internal mark,mid term mark,final mark,total
0,Harper Lee,Object Oriented programming,15,20,35,70.0
1,Garcia Marquez,Data Structure,17,15,30,62.0
2,"Forster, E.M.",Object Oriented programming,18,23,38,79.0
3,Ralph Ellison,Unix programming,20,22,40,82.0


In [ ]:
# for bigquery

-- geting data from students table and courses table

SELECT
      students.name AS name,
      courses.name AS course,
      MAX(CASE WHEN result.mark_type = "Internal Exam"
                THEN marks
          END) internal_mark,
      MAX( CASE WHEN result.mark_type = "Mid Term Exam"
                THEN marks
            END) mid_term_exam,
      MAX( CASE WHEN result.mark_type = "Final Exam"
                THEN marks
            END) final_mark
from
    `data-sci-acad-learn-sql.TW14_N10.result`  AS result,
    `data-sci-acad-learn-sql.TW14_N10.students` AS students,
    `data-sci-acad-learn-sql.TW14_N10.courses` AS courses
WHERE
    result.std_id = students.id
    AND
    result.course_id = courses.course_id
GROUP BY
    result.std_id,
    result.course_id,
    students.name,  #! have to group by this
    courses.name     #! have to group by this